In [1]:
# Librerias y configuración

import pandas as pd
import numpy as np
from tqdm import tqdm
from pycaret.regression import *
from sklearn.preprocessing import StandardScaler

In [2]:
# Cargar datos

df = pd.read_csv('../data/complete_data.csv')

df.head()

,fecha,intensidad,estacion,magnitud,unidad_medida,valores
0,2022-12-01 00:00:00,222.189757,Pza. de España,Dióxido de Azufre,ug/m3,1.0
1,2022-12-01 00:00:00,222.189757,Arturo Soria,Óxidos de Nitrógeno,ug/m3,18.0
2,2022-12-01 00:00:00,222.189757,Barajas,Dióxido de Nitrógeno,ug/m3,26.0
3,2022-12-01 00:00:00,222.189757,Barrio del Pilar,Óxidos de Nitrógeno,ug/m3,17.0
4,2022-12-01 00:00:00,222.189757,Pza. Castilla,Óxidos de Nitrógeno,ug/m3,16.0


In [3]:
# Se hace una lista con un df por cada contaminante para poder hacer un modelo por cada uno

# Primero se crea una columna para cada contaminante con su valor

df_magnitudes = df.pivot_table(index='fecha', columns='magnitud', values='valores')

df_magnitudes = df_magnitudes.fillna(method='ffill')

# Se elimminan las columnas magnitud y valores del df original porque ya no se necesitan

df = df.drop(columns=['magnitud', 'valores'])

# Se hace un merge con el df original

df = pd.merge(df, df_magnitudes, on='fecha')

# Se reduce el df para ajustar el número de datos a la capacidad de la máquina, en este caso 0.1% de los datos puesto que por ahora solo se dispone de datos de tráfico de Madrid y es una prueba, pero se debe cambiar en caso de ser posible

muestra_aleatoria = np.random.choice(df.index, size=int(len(df) * 0.00001), replace=False)

df = df.loc[muestra_aleatoria]

# Se hace un df con la media de contaminación en toda la ciudad, no por estación

df = df.groupby('fecha').mean().reset_index()

# Se crea la lista donde se guardarán los dataframes

dfs = []

# Se quiere hacer un ciclo for con cada columna de contaminante

for e in tqdm(df[['Benceno',
       'Dióxido de Azufre', 'Dióxido de Nitrógeno', 'Etilbenceno',
       'Hidrocarburos no metánicos', 'Hidrocarburos totales', 'Metano',
       'Monóxido de Carbono', 'Monóxido de Nitrógeno', 'Ozono',
       'Óxidos de Nitrógeno', 'Partículas < 10 um', 'Partículas < 2.5 um',
       'Tolueno']].columns):

    # Después se hace un df con la media de contaminación en la ciudad por cada contaminante

    variables = ['intensidad', e]   # Posibles variables explicativas + los valores del contaminante. Añafir aqui nuevas columnas si se añaden nuevas variables/datos

    df_magnitudes = df[variables]

    # Se añade el df a la lista de dataframes

    dfs.append(df_magnitudes)

100%|██████████| 14/14 [00:00<00:00, 858.01it/s]


In [4]:
dfs[0].head()

,intensidad,Benceno
0,873.370061,0.466667
1,772.634227,0.616667
2,750.996914,0.675000
3,347.174069,0.250000
4,77.778486,0.500000


In [7]:
# Se usa pycaret para comparar los distintos modelos de regresión y conocer sus resultados

# Se crea un objeto de la clase StandardScaler para normalizar los datos

scaler = StandardScaler() 

for e in tqdm(dfs):

    print(f'CONTAMINANTE: {e.columns[1]}')   # Cambiar el número de columna en caso de añadir nuevas variables/datos

    # Se normalizan los datos

    e = pd.DataFrame(scaler.fit_transform(e))
    
    reg = setup(data=e, target=f'{e.columns[1]}', session_id=1)  # Cambiar el número de columna en caso de añadir nuevas variables/datos

    compare_models()

    print('\n\n\n\n\n\n') # Unos saltos de linea para que sea más legible

  0%|          | 0/14 [00:00<?, ?it/s]

CONTAMINANTE: Benceno


,Description,Value
0,Session id,1
1,Target,1
2,Target type,Regression
3,Original data shape,"(98, 2)"
4,Transformed data shape,"(98, 2)"
5,Transformed train set shape,"(68, 2)"
6,Transformed test set shape,"(30, 2)"
7,Numeric features,1
8,Preprocess,True
9,Imputation type,simple


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:12:04
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.7136,1.4081,1.0007,-0.4311,0.4099,1.7677,0.0120
llar,Lasso Least Angle Regression,0.8069,1.3237,0.9944,-0.5007,0.5001,1.3546,0.0110
lasso,Lasso Regression,0.8069,1.3237,0.9944,-0.5007,0.5001,1.3546,0.0190
dummy,Dummy Regressor,0.8069,1.3237,0.9944,-0.5007,0.5001,1.3546,0.0110
en,Elastic Net,0.8069,1.3237,0.9944,-0.5007,0.5001,1.3546,0.0150
lightgbm,Light Gradient Boosting Machine,0.7724,1.2612,0.9658,-0.5178,0.4051,2.1562,0.0130
knn,K Neighbors Regressor,0.8171,1.3444,1.0283,-0.6703,0.3891,2.1593,0.0130
br,Bayesian Ridge,0.8107,1.3305,1.0073,-0.7598,0.5101,1.7354,0.0120
ridge,Ridge Regression,0.7983,1.3241,1.0035,-0.8785,0.4881,1.9297,0.0150
lr,Linear Regression,0.7984,1.3256,1.0043,-0.8921,0.4876,1.9409,0.0140


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.5938,1.4395,0.9703,-1.2674,0.3520,1.0480,0.0120
llar,Lasso Least Angle Regression,0.7421,1.3736,0.9721,-1.3758,0.4570,1.2339,0.0120
lasso,Lasso Regression,0.7421,1.3736,0.9721,-1.3758,0.4570,1.2339,0.0120
dummy,Dummy Regressor,0.7421,1.3736,0.9721,-1.3758,0.4570,1.2339,0.0120
en,Elastic Net,0.7421,1.3736,0.9721,-1.3758,0.4570,1.2339,0.0150
lightgbm,Light Gradient Boosting Machine,0.7141,1.3482,0.9687,-3.1153,0.3589,1.7557,0.0150
br,Bayesian Ridge,0.7546,1.4070,1.0084,-4.4096,0.4651,1.4258,0.0130
knn,K Neighbors Regressor,0.7493,1.4415,1.0150,-4.6228,0.3674,1.9853,0.0130
ridge,Ridge Regression,0.7427,1.3970,1.0044,-5.3273,0.4424,1.4942,0.0120
lr,Linear Regression,0.7431,1.3995,1.0057,-5.4358,0.4420,1.4993,0.0130


 79%|███████▊  | 11/14 [01:27<00:23,  7.90s/it]








CONTAMINANTE: Partículas < 10 um


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.7050,1.3956,1.0482,-0.2917,0.4342,3.1709,0.0130
lasso,Lasso Regression,0.7599,1.2964,1.0346,-0.3234,0.4874,1.5254,0.0120
en,Elastic Net,0.7599,1.2964,1.0346,-0.3234,0.4874,1.5254,0.0140
dummy,Dummy Regressor,0.7599,1.2964,1.0346,-0.3234,0.4874,1.5254,0.0110
llar,Lasso Least Angle Regression,0.7599,1.2964,1.0346,-0.3234,0.4874,1.5254,0.0120
br,Bayesian Ridge,0.7740,1.3313,1.0498,-0.3799,0.4869,1.4943,0.0150
ridge,Ridge Regression,0.7637,1.3257,1.0487,-0.4496,0.4895,1.5882,0.0120
lar,Least Angle Regression,0.7640,1.3269,1.0493,-0.4541,0.4893,1.5911,0.0170
omp,Orthogonal Matching Pursuit,0.7640,1.3269,1.0493,-0.4541,0.4893,1.5911,0.0120
lr,Linear Regression,0.7640,1.3269,1.0493,-0.4541,0.4893,1.5911,0.0130


100%|██████████| 14/14 [01:51<00:00,  7.94s/it]